In [483]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# import ollama
import time
import pandas as pd

In [484]:
chrome_options = Options()

chrome_options.add_argument("--disable-bot-detection")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36") # Anti-bot evasion techniques
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)


In [485]:
# Initialize WebDriver
options = Options()
options.binary_location = "/snap/bin/chromium"
service = Service(executable_path='/snap/bin/chromium.chromedriver')
driver = webdriver.Chrome(service=service, options=chrome_options)


In [486]:
# 3. Configure Wait Strategy
wait = WebDriverWait(driver, 120)
try:
    # 4. Load page with JavaScript verification
    url = "https://www.oddsportal.com/football/england/premier-league-2014-2015/results/#/page/8/"
    driver.get(url)
    
    # 5. Add human-like delay
    time.sleep(6)  # Let initial JavaScript execute
    
    # 6. Alternative waiting strategies
    # Wait for either the table OR a loading element to disappear
    wait.until(
        EC.any_of(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table.table-main")),
            EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.loading")),
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.eventRow"))
            
        )
    )
    
    # 7. Additional verification
    if "oddsportal.com" not in driver.current_url:
        print("Redirect detected! Current URL:", driver.current_url)
    
    # Scroll to load all matches (adjust based on your needs)
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    
except Exception as e:
    print(f"Error processing row: {e}")
            

In [487]:
# Convert to uk
def american_to_decimal(odds):
    try:
        if odds.startswith('+'):
            decimal_odds = 1 + int(odds[1:]) / 100
            return round(decimal_odds, 2)
        elif odds.startswith('-'):
            decimal_odds = 1 + 100 / abs(int(odds[1:]))
            return round(decimal_odds, 2)
        return round(float(odds), 2)
    except (ValueError, TypeError, ZeroDivisionError):
        return odds

# Extract match data with winning odd
try: 
    matches = []
    event_rows = driver.find_elements(By.CSS_SELECTOR, "div.eventRow")

    for row in event_rows:
        try:
            # Extract team names
            participants = row.find_elements(By.CSS_SELECTOR, "a[title]")
            if len(participants) < 2:
                continue
                
            home_team = participants[0].get_attribute("title")
            away_team = participants[1].get_attribute("title")
            
            # Extract odds containers (all three outcomes)
            odds_containers = row.find_elements(
                By.CSS_SELECTOR, "div[data-testid^='odd-container']"
            )
            
            if len(odds_containers) < 3:
                continue
                
            # Initialize variables
            home_odds = None
            draw_odds = None
            away_odds = None
            winning_odd = None
            winning_outcome = None  # "Home", "Draw", or "Away"

            # Process each odds container
            for i, container in enumerate(odds_containers[:3]):
                # Find the <p> tag containing the odd value
                p_tag = container.find_element(By.CSS_SELECTOR, "p")
                odd_value = american_to_decimal(p_tag.text)
                
                # Determine if this is the winning odd
                if p_tag.get_attribute("data-testid") == "odd-container-winning":
                    winning_odd = odd_value
                    winning_outcome = ["Home", "Draw", "Away"][i]
                    
                # Assign to corresponding outcome
                if i == 0:
                    home_odds = odd_value
                elif i == 1:
                    draw_odds = odd_value
                elif i == 2:
                    away_odds = odd_value
                    
            matches.append({
                "Home Team": home_team,
                "Away Team": away_team,
                "Home Odds": home_odds,
                "Draw Odds": draw_odds,
                "Away Odds": away_odds,
                "Winning Outcome": winning_outcome,
                "Winning Odd": winning_odd,
                "league": "English Premier League"
            })
        except Exception as e:
                print(f"Error processing row: {e}")
                continue
finally:
    driver.quit()

In [488]:
# Save to DataFrame
df = pd.DataFrame(matches)

# Create DataFrame
if matches:
    df = pd.DataFrame(matches)
    # Save to CSV
    df.to_csv("premier_league_result_odds.csv", index=False, mode='a',header=False)
    print("\nData saved to premier_league_result_odds.csv")
else:
    print("No matches found")



Data saved to premier_league_result_odds.csv
